# Competitor Price Monitoring Bot
### I developed an automated script in Python (BeautifulSoup) that extracts pricing and availability data from e-commerce websites. This tool allows marketing teams to monitor the competition in real time without manual review.

In [ ]:
!pip install requests beautifulsoup4

In [ ]:
# --- LIBRARIES ---
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# --- CONFIGURATION ---
base_url = "http://books.toscrape.com/catalogue/page-{}.html"
pages_to_scrape = 5
all_books_data = [] # Master list to store data

print(f"Starting scraper... Target: {pages_to_scrape} pages.\n")

# --- PAGINATION LOOP ---
for page_number in range(1, pages_to_scrape + 1):

  # 1. Construct the Dynamic URL
  target_url = base_url.format(page_number)
  print(f"Scraping page {page_number} - URL: {target_url}")

  # 2. Send Request
  try:
    response = requests.get(target_url)

    # Check if the request was successful (status 200)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')

      # find all book containers on this page
      book_containers = soup.find_all('article', class_='product_pod')

      # 3. Extract data for each book
      for book in book_containers:
        # extract title
        title = book.find('h3').find('a')['title']

        # extract price and clean currency symbol
        price_text = book.find('p', class_='price_color').text
        price = price_text.replace('Â£', '£')

        # extract availability
        stock = book.find('p', class_='instock availability').text.strip()

        # and then we append to master list
        all_books_data.append({
          'Title': title,
          'Price': price,
          'stock': stock
        })

    else:
            print(f"⚠️ Failed to retrieve Page {page_number}. Status: {response.status_code}")

  except Exception as e:
    print(f"⚠️ Error on Page {page_number}: {e}")

# 4. POLITENESS DELAY (Crucial for avoiding blocks)
    # We wait 1 second between pages to respect the server
  time.sleep(1)

# --- SAVE & EXPORT ---
print(f"\n Task Completed! Successfully extracted {len(all_books_data)} books.")

# Convert to DataFrame
df_books = pd.DataFrame(all_books_data)

# Display first and last rows to verify pagination worked
display(df_books.head())
display(df_books.tail())

Starting scraper... Target: 5 pages.

Scraping page 1 - URL: http://books.toscrape.com/catalogue/page-1.html
Scraping page 2 - URL: http://books.toscrape.com/catalogue/page-2.html
Scraping page 3 - URL: http://books.toscrape.com/catalogue/page-3.html
Scraping page 4 - URL: http://books.toscrape.com/catalogue/page-4.html
Scraping page 5 - URL: http://books.toscrape.com/catalogue/page-5.html

 Task Completed! Successfully extracted 100 books.


,Title,Price,stock
0,A Light in the Attic,£51.77,In stock
1,Tipping the Velvet,£53.74,In stock
2,Soumission,£50.10,In stock
3,Sharp Objects,£47.82,In stock
4,Sapiens: A Brief History of Humankind,£54.23,In stock


,Title,Price,stock
95,Lumberjanes Vol. 3: A Terrible Plan (Lumberjan...,£19.92,In stock
96,"Layered: Baking, Building, and Styling Spectac...",£40.11,In stock
97,Judo: Seven Steps to Black Belt (an Introducto...,£53.90,In stock
98,Join,£35.67,In stock
99,In the Country We Love: My Family Divided,£22.00,In stock
